In [2]:
# requests.__version__
# '2.25.1'

# pd.__version__
# '1.2.3'

'2.25.1'

In [46]:
import requests
import pandas as pd
from os import path, makedirs
import re
from collections import OrderedDict
from datetime import datetime

In [57]:
## lists and dicts config deleted from here ##

def pandasRegion(DataFrame, SourceColumn, TargetColumn, Reference):
    def insideRegion(pd_df):
        g = pd_df[SourceColumn]
        if g in Reference:
            return 'Yes'
        else:
            return 'No'
    DataFrame[TargetColumn] = DataFrame.apply(insideRegion, axis=1)
    return DataFrame

def areaName(DataFrame, SourceColumn, TargetColumn, Reference):
    def nameLookup(pd_df):
        f = pd_df[SourceColumn]
        if f in Reference:
            n = Reference[f]
            return n
        else:
            return None
    DataFrame[TargetColumn] = DataFrame.apply(nameLookup, axis=1)
    return DataFrame

In [ ]:
# Set year, key

y = 2020
mrcogkey = '722d2ba6712f3952d8daba907b503768680f20ce'

# Set working paths

working_folder = path.normpath('D:/...........................................................')

# Get list of tables
tables = pd.read_excel(
    path.join(working_folder, 'ACS_Tables_for_Download_python.xlsx'),
    header=None,
    sheet_name=0,
    usecols=0,
    squeeze=True
).to_list()

# def getUniversesFromShells()
# def getUniversesFromOtherTable()
# def getVariablesByYear()
# def getConcept()

# Supp resources
## Field/alias dictionary
## Universes
## Table/file names

# For table in table list...

###### For tract AND blockgroup...

## Query to data frame

'''COG FORMATTING'''
## !!Before sorting columns, need to add columns for region
'''Sorting columns should be a function. Then it can be used in the main DF function to make numeric,
and then be called by the user for annotation, etc.'''

## def simpleGeoid()

## !!In order to get annotation, need to sort columns
## Split out annotation >>> Save
        # Is annotation saved in the same data dictionary format? Or like an AFF table?
## !!!! Probably need a folder for annotation
        # Could the empty table check be used for annotation? I think so?
    
    
# After that, the rest is just formatting? Table name, dictionary, and universe?

In [91]:
testQuery = 'https://api.census.gov/data/2017/acs/acs5?get=group(B05002)&for=tract:*&in=state:35&in=county:*&key=722d2ba6712f3952d8daba907b503768680f20ce'

testCall = requests.get(testQuery)

testCall, testCall.status_code, type(testCall), type(testCall.status_code)

(<Response [200]>, 200, <class 'requests.models.Response'>, <class 'int'>)

In [92]:
j = testCall.json()

df = pd.DataFrame(j[1:], columns=j[0])

dfdrop = df.dropna(axis='columns', how='all')

dfdrop.shape, df.shape, dfa.shape

[['B05002_001E', 'B05002_001EA', 'B05002_001M', 'B05002_001MA', 'B05002_002E', 'B05002_002EA', 'B05002_002M', 'B05002_002MA', 'B05002_003E', 'B05002_003EA', 'B05002_003M', 'B05002_003MA', 'B05002_004E', 'B05002_004EA', 'B05002_004M', 'B05002_004MA', 'B05002_005E', 'B05002_005EA', 'B05002_005M', 'B05002_005MA', 'B05002_006E', 'B05002_006EA', 'B05002_006M', 'B05002_006MA', 'B05002_007E', 'B05002_007EA', 'B05002_007M', 'B05002_007MA', 'B05002_008E', 'B05002_008EA', 'B05002_008M', 'B05002_008MA', 'B05002_009E', 'B05002_009EA', 'B05002_009M', 'B05002_009MA', 'B05002_010E', 'B05002_010EA', 'B05002_010M', 'B05002_010MA', 'B05002_011E', 'B05002_011EA', 'B05002_011M', 'B05002_011MA', 'B05002_012E', 'B05002_012EA', 'B05002_012M', 'B05002_012MA', 'B05002_013E', 'B05002_013EA', 'B05002_013M', 'B05002_013MA', 'B05002_014E', 'B05002_014EA', 'B05002_014M', 'B05002_014MA', 'B05002_015E', 'B05002_015EA', 'B05002_015M', 'B05002_015MA', 'B05002_016E', 'B05002_016EA', 'B05002_016M', 'B05002_016MA', 'B0500

In [98]:
df['GEO_ID'] = df['GEO_ID'].str[9:]

# Adding MRCOG column County name columns should come before sorting columns
df = areaName(df, 'county', 'CountyName', new_mexico_fips)
df = pandasRegion(df, 'GEO_ID', 'MRCOG_Area', mrcog_tracts)

In [100]:
colCodes = df.columns.to_list()

aCols, eCols, mCols, placeCols = [], [], [], []
# Sort the codes into separate lists
for c in colCodes:
    if c[-1] == 'A':
        aCols.append(c)
    elif c[-1] == 'E' and c != 'NAME':
        eCols.append(c)
    elif c[-1] == 'M':
        mCols.append(c)
    else:
        placeCols.append(c)

# annoCols = [c for c in colCodes if c[-1] == 'A']
# numCols = [c for c in colCodes if c[-1] in ['E', 'M']]

In [101]:
txtToNumCols = eCols + mCols
txtToNumCols.sort()
for col in txtToNumCols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

colCodes = placeCols + txtToNumCols

annoSchema = placeCols + aCols

dfanno = df[annoSchema]

df = df[colCodes]

df.shape

(499, 61)

In [102]:
# dfa = df1.loc[:, [i for i in df1.columns if i[-1]=='A']]

dfanno.sample(5)

,GEO_ID,NAME,state,county,tract,CountyName,MRCOG_Area,B05002_001EA,B05002_001MA,B05002_002EA,B05002_002MA,B05002_003EA,B05002_003MA,B05002_004EA,B05002_004MA,B05002_005EA,B05002_005MA,B05002_006EA,B05002_006MA,B05002_007EA,B05002_007MA,B05002_008EA,B05002_008MA,B05002_009EA,B05002_009MA,B05002_010EA,B05002_010MA,B05002_011EA,B05002_011MA,B05002_012EA,B05002_012MA,B05002_013EA,B05002_013MA,B05002_014EA,B05002_014MA,B05002_015EA,B05002_015MA,B05002_016EA,B05002_016MA,B05002_017EA,B05002_017MA,B05002_018EA,B05002_018MA,B05002_019EA,B05002_019MA,B05002_020EA,B05002_020MA,B05002_021EA,B05002_021MA,B05002_022EA,B05002_022MA,B05002_023EA,B05002_023MA,B05002_024EA,B05002_024MA,B05002_025EA,B05002_025MA,B05002_026EA,B05002_026MA,B05002_027EA,B05002_027MA
337,35001004752,"Census Tract 47.52, Bernalillo County, New Mexico",35,001,004752,Bernalillo County,Yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
233,35045942802,"Census Tract 9428.02, San Juan County, New Mexico",35,045,942802,San Juan County,No,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
269,35001000601,"Census Tract 6.01, Bernalillo County, New Mexico",35,001,000601,Bernalillo County,Yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
135,35043940600,"Census Tract 9406, Sandoval County, New Mexico",35,043,940600,Sandoval County,Yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
70,35001000109,"Census Tract 1.09, Bernalillo County, New Mexico",35,001,000109,Bernalillo County,Yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [103]:
df = df.replace(annotation_all_types, '')

In [ ]:
# # Base API URL
# base_url = 'https://api.census.gov/data/{}/acs/acs5'.format(yr)
# # subject_base_url = '{}/subject'.format(base_url)

# # Variables (table codes and names as dictionary)
# variables_url = '{}/variables.json'.format(base_url)

# # Request for address, returned as text
# get_variables = requests.get(variables_url)

# # Text to JSON (python recognizes as a dictionary)
# variableDict = get_variables.json()

dataProductList_5Year_xlsx = '2020_DataProductList_5Year.xlsx'
dpl = pd.read_excel(dataProductList_5Year_xlsx)
tableTitles = dict(zip(dpl['Table ID'], dpl['Table Title']))
tableUniverses = dict(zip(dpl['Table ID'], dpl['Table Universe']))

tableFileNames = {k: REMOVESPECIALCHARACTERS(v) for k, v in tableTitles.items()}

universes = {k: v.replace('Universe: ', '') for k, v in tableUniverses.items()}



"""MAKE ME A FUNCTION"""
# plain_concept = variableDict['variables'][
#         '{}_001E'.format(code)]['concept'].title()
# Prepare a file system compliant table name by removing special chars
# and shortening where possible
concept = plain_concept.translate({
        ord(c): '' for c in '''!"'(),-./:;[]{}'''
        })
filename_dict = OrderedDict({
        f'In {YEAR YEAR YEAR YEAR} InflationAdjusted Dollars': '',
        ' ': '_',
        'The_United_States': 'US',
        'In_The_Past_12_Months': 'In_Past_Year',
        'The_': '',
        '___': '_',
        '__': '_'
        })
for k, v in filename_dict.items():
    concept = concept.replace(k, v)
    
    
    
    
    
    
# Make field Alias dictionary instead of processColumns
# This needs to account for how the variables page shows "Other" fields
# This is similar to what occurs below. Update names to reflect field name, field alias


# Use the function to begin processing column (field) names
result = processColumns(result)
# Get a list of column codes/names for further processing
colCodes = result.columns.tolist()
# This ordered dictionary, with column codes as keys,
# forms the basis of a data dictionary for each table
fielDict = OrderedDict.fromkeys(colCodes)
# Iterate through keys, adding values based on returns from the func
# The data dictionary will be a table with 3 columns:
# Code, plain text, and field name
for f in fielDict:
    fielDict[f] = columnNames(f)
labels_list = [i[1] for i in fielDict.values()]
# The "result" Data Frame will become the Excel sheet and GDB table
# It needs clear but database compliant field names
result.columns = labels_list

In [112]:
# x = dict(zip(df.GEO_ID, zip(df.state, df.county)))
# x



{'35003976400': 'S', '35019961600': 'o', '35033955200': 'm', '35059950200': 'e', '35001004716': '0', '35001004746': ' ', '35001000124': ' ', '35001000116': ' ', '35061970902': ' ', '35001003726': ' ', '35043010705': ' ', '35045000707': '0', '35049000101': '0', '35001000127': '3', '35001004713': '\n', '35001001700': '1', '35006945800': ' ', '35045943201': ' ', '35001004729': ' ', '35001003202': ' ', '35009000202': ' ', '35001003717': ' ', '35013001600': '0', '35025000503': '1', '35031940500': '9', '35009000400': '\n', '35023970000': '2', '35043010712': ' ', '35041000402': ' ', '35015001000': ' ', '35015001100': ' ', '35047957500': ' ', '35031943500': ' ', '35013001306': '0', '35017964300': '3', '35017964600': '3', '35035000100': '\n', '35001004502': '3', '35013000401': ' ', '35031946000': ' ', '35027960300': ' ', '35043010720': ' ', '35001000502': ' ', '35005000900': ' ', '35049001102': '0', '35045000205': '5', '35037958601': '9', '35043940500': '\n', '35043010503': '4', '35043010721': 

In [111]:
'abc'.title()

'Abc'

In [114]:
{'ab{}ef'.format('CD'): 23}

{'abCDef': 23}